# Example: Lattice Model of Interest Rates
Fill me in

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [6]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2

## Prerequisite: Load the Treasury Rates Dataset
Fill me in

## Task 1: Build a Lattice Model and Look at its properties
Fill me in

In [13]:
p = 0.50;
u = 1.25;
d = 1/u;
T = 2; # this will give T + 1 levels (zero based)
rₒ = 0.1;

Fill me in

In [17]:
my_lattice_tree_model = build(MySymmetricBinaryInterestRateLatticeModel, (
    p = p, u = u, d = d, T = T, rₒ = rₒ
)) |> populate;

Fill me in

In [19]:
my_lattice_tree_model.data

Dict{Int64, MyBinaryInterestRateLatticeNodeModel} with 6 entries:
  0 => MyBinaryInterestRateLatticeNodeModel(1.0, 0.1, 0.0)
  4 => MyBinaryInterestRateLatticeNodeModel(0.5, 0.1, 2.3834e-314)
  5 => MyBinaryInterestRateLatticeNodeModel(0.25, 0.064, 3.15506e-310)
  2 => MyBinaryInterestRateLatticeNodeModel(0.5, 0.08, 2.3834e-314)
  3 => MyBinaryInterestRateLatticeNodeModel(0.25, 0.15625, 2.3834e-314)
  1 => MyBinaryInterestRateLatticeNodeModel(0.5, 0.125, 2.3834e-314)

Fill me in

In [21]:
my_lattice_tree_model.connectivity

Dict{Int64, Vector{Int64}} with 3 entries:
  0 => [1, 2]
  2 => [4, 5]
  1 => [3, 4]

Fill me in

In [23]:
my_lattice_tree_model.levels

Dict{Int64, Vector{Int64}} with 3 entries:
  0 => [0]
  2 => [3, 4, 5]
  1 => [1, 2]

## Task 2: Calibrate the Lattice Model Parameters
Fill me in

## Task 3: Zero Price Simulation
Fill me in

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.